In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.append('../ranewable/')
from ranewable import Ra

/anaconda3/envs/power-forecast/lib/python3.8/site-packages/pvlib/forecast.py:18: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  warnings.warn(


## Load GEFCom2014 data

In [2]:
# Solar farm data
farm_coords = [[145, -37,5], [145, -37,5], [145, -37,5]]
farm_altitude = [595, 602, 951]
panel_orientation = [38, 327, 31]
panel_tilt = [36, 35, 21]

In [5]:
df = pd.read_csv('../data/gefcom2014/raw/gefcom2014-solar-raw.csv', index_col=0, header=[0,1], parse_dates=True)
features_accum = ['VAR169', 'VAR175', 'VAR178', 'VAR228']
df_accum = df.loc[:,(slice(None),features_accum)]
df_accum = df_accum.diff()
df_accum[df_accum.index.hour==0] = df.loc[df_accum.index.hour==0,(slice(None),features_accum)]
df_accum.loc[:,(slice(None),features_accum[:3])] = df_accum.loc[:,(slice(None),features_accum[:3])]/(3600) # Convert from J to kWh/h
df.loc[:,(slice(None),features_accum)] = df_accum

In [6]:
df.head()

ZONEID,1,2,3,1,2,3,1,2,3,1,...,3,1,2,3,1,2,3,1,2,3
,POWER,POWER,POWER,VAR134,VAR134,VAR134,VAR157,VAR157,VAR157,VAR164,...,VAR178,VAR228,VAR228,VAR228,VAR78,VAR78,VAR78,VAR79,VAR79,VAR79
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2012-04-01 01:00:00,0.754103,0.647328,0.799000,94843.6250,94290.6250,92667.6250,60.221909,57.374252,57.003159,0.244601,...,NaN,NaN,NaN,NaN,0.001967,0.001479,0.003531,0.003609,0.005444,0.024673
2012-04-01 02:00:00,0.555000,0.621822,0.817500,94757.9375,94217.6875,92622.6875,54.678604,57.129776,64.960831,0.457138,...,855.296944,0.000000,0.000000,0.000774,0.005524,0.018304,0.027482,0.033575,0.101553,0.140474
2012-04-01 03:00:00,0.438397,0.472692,0.571500,94732.8125,94201.0625,92598.8125,61.294891,63.076141,66.795868,0.771429,...,789.361667,0.001341,0.001256,0.000898,0.030113,0.035255,0.032356,0.132009,0.112935,0.136617
2012-04-01 04:00:00,0.145449,0.098866,0.063775,94704.0625,94156.0625,92542.0625,67.775284,62.975479,65.177628,0.965866,...,654.504167,0.001161,0.000469,0.000413,0.057167,0.064514,0.031952,0.110645,0.107761,0.097370
2012-04-01 05:00:00,0.111987,0.104393,0.103700,94675.0000,94124.2500,92508.0000,70.172989,65.053848,65.744278,0.944669,...,528.086389,0.000830,0.000367,0.000450,0.051027,0.059495,0.048494,0.189560,0.157593,0.163773


## Create ranewable instance

In [7]:
ra = Ra(longitude=farm_coords[0][0],
        latitude=farm_coords[0][1],
        altitude=farm_altitude[0],
        capacity=1,
        orientation=panel_orientation[0],
        tilt=panel_tilt[0])

In [9]:
weather = ra.weather_from_ghi(df['1']['VAR169'])

In [12]:
pred = ra.calculate_power(weather)

/anaconda3/envs/power-forecast/lib/python3.8/site-packages/pvlib/modelchain.py:956: pvlibDeprecationWarning: times keyword argument is deprecated and will be removed in 0.8. The index of the weather DataFrame is used for times.
  warnings.warn('times keyword argument is deprecated and will be '


In [13]:
df.loc[:, ('1', 'PRED')] = pred

In [14]:
abs_error = (df['1']['POWER']-df['1']['PRED']).abs().mean()

In [15]:
# compare result with other notebook using pvlib directly
print(abs_error)
print(0.08764306544934268)

0.07749390373561786
0.08764306544934268
